# Deep Neural Networks

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
# Python 2 and 3 Compatibility Library
from six.moves import cPickle as pickle

In [3]:
pickle_file = "data/notMNIST.pickle"

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
  
    train_data_raw = save['train_dataset']
    train_target_raw = save['train_labels']
  
    valid_data_raw = save['valid_dataset']
    valid_target_raw = save['valid_labels']
  
    test_data_raw = save['test_dataset']
    test_target_raw = save['test_labels']
  
    del save  

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [4]:
def reformat(dataset, labels):
  
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [5]:
def output_size_pool(input_size, conv_filter_size, pool_filter_size, 
                                    padding, conv_stride, pool_stride):
    if padding == 'SAME':
        padding = -1.00
    elif padding == 'VALID':
        padding = 0.00
    else:
        return None
    
    output_1 = (((input_size - conv_filter_size - 2*padding)/ conv_stride) + 1.00)
    print('output1: ', output_1)  
    
    output_2 = (((output_1 - pool_filter_size - 2*padding)  / pool_stride) + 1.00)    
    print('output2: ', output_2)
    
    output_3 = (((output_2 - conv_filter_size - 2*padding)  / conv_stride) + 1.00)
    print('output3: ', output_3) 
    
    output_4 = (((output_3 - pool_filter_size - 2*padding)  / pool_stride) + 1.00)  
    print('output4: ', output_4)
    
    return int(output_4)


### IPERPARAMETRI

In [6]:
#
# INPUT
#
# La dimensione delle immagini 28x28
image_size = 28
# la dimensione della colonna target 10x1
num_labels = 10
# La profondita' dei dati in input
num_channels = 1 
# 1 = scala di grigi

#
# DATASET 
#
train_data, train_target = reformat(train_data_raw, train_target_raw)
valid_data, valid_target = reformat(valid_data_raw, valid_target_raw)
test_data,   test_target = reformat(test_data_raw,   test_target_raw)

#
# FILTERS
#
# Dimensione del minibatch
batch_size = 16
# Dimensione della finestra convoluzione 5x5
patch_size = 5
# Numero di features (o profondita') del filtro 
depth = 32
# Nodi dell'hidden layer
num_hidden = 64
# Numero di celle che il filtro scorre 
stride = 1
# Dimensione della finestra pooling 2x2
pool_size = 2 
# Numero di celle che il pooling scorre 
pool_stride = 2
# Probabilita' del Dropout
keep_prob = 0.5

In [7]:
x = output_size_pool(image_size, patch_size, pool_size, 'VALID', stride, pool_stride)

output1:  24.0
output2:  12.0
output3:  8.0
output4:  4.0


### ARCHITETTURA TENSORFLOW

IN -> CNN -> RELU + POOL -> CNN -> RELU + POOL -> 

-> RESHAPE -> FULL + RELU + DROPOUT -> FULL + RELU + DROPOUT -> FULL(OUT)

In [67]:
graph = tf.Graph()

with graph.as_default():
    
    # Per i minibatch utilizzo placeholder
    tf_train_data = tf.placeholder(tf.float32,
            shape=(batch_size, image_size, image_size, num_channels))
    tf_train_target = tf.placeholder(tf.float32,
            shape=(batch_size,                         num_labels))
    
    # Senza minibatch
    tf_valid_data = tf.constant(valid_data)
    tf_test_data  = tf.constant(test_data)
    
    # Layer1 CNN + RELU + POOL
    # Dimens   = l_finestra, h_finestra, prof_input, prof_output
    dim_layer1 = [patch_size, patch_size, num_channels, depth]
    # Pesi RANDOM + TRUNCATE valori troppo bassi o alti
    tf_layer1_pesi = tf.Variable(
                tf.truncated_normal(dim_layer1, stddev=0.1))
    # Bias RANDOM (0)
    tf_layer1_bias = tf.Variable(tf.zeros([depth]))
    
    # Layer2 CNN + RELU + POOL
    # Dimens   = l_finestra, h_finestra, prof_input, prof_output
    dim_layer2 = [patch_size, patch_size, depth, depth]
    # Pesi RANDOM + TRUNCATE 
    tf_layer2_pesi = tf.Variable(
                tf.truncated_normal(dim_layer2, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer2_bias = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Layer3 FULL
    # Dimens   = 2CNN+2POOL * 2CNN+2POOL * prof_in, prof_ou
    dim_layer3 = [x * x * depth, num_hidden]
    # Pesi RANDOM + TRUNCATE 
    tf_layer3_pesi = tf.Variable(
                tf.truncated_normal(dim_layer3, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer4 FULL
    # Dimens   = lunghezza, altezza
    dim_layer4 = [num_hidden, num_hidden]
    # Pesi RANDOM + TRUNCATE 
    tf_layer4_pesi = tf.Variable(
                tf.truncated_normal(dim_layer4, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer4_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer5 FULL
    # Dimens   = lunghezza, altezza
    dim_layer5 = [num_hidden, num_labels]
    # Pesi RANDOM + TRUNCATE 
    tf_layer5_pesi = tf.Variable(
                tf.truncated_normal(dim_layer5, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer5_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    #
    # MODELLO
    # 
    def model(data):
        # Convolutional LAYER1
        conv_1 = tf.nn.conv2d(data, tf_layer1_pesi, 
                              strides=[1, stride, stride, 1], 
                              padding='VALID')
        # RELU LAYER1
        hidden_1 = tf.nn.relu(conv_1 + tf_layer1_bias)
        # Average POOLING LAYER1
        pool_1 = tf.nn.avg_pool(hidden_1, 
                                [1, pool_size, pool_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Convolutional LAYER2
        conv_2 = tf.nn.conv2d(pool_1, tf_layer2_pesi, 
                              strides=[1, stride, stride, 1], 
                              padding='VALID')
        # RELU LAYER2
        hidden_2 = tf.nn.relu(conv_2 + tf_layer2_bias)
        # Average POOLING LAYER2
        pool_2 = tf.nn.avg_pool(hidden_2, 
                                [1, pool_size, pool_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Full Connected LAYER3
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, 
                            [shape[0], shape[1] * shape[2] * shape[3]])
        # RELU LAYER3
        hidden_3 = tf.nn.relu(tf.matmul(reshape, 
                                      tf_layer3_pesi) + tf_layer3_bias)
        # DROPOUT LAYER3
        hidden_drop_3 = tf.nn.dropout(hidden_3, keep_prob)
        
        # Full Connected RELU LAYER4
        hidden_4 = tf.nn.relu(tf.matmul(hidden_drop_3, 
                                        tf_layer4_pesi) + tf_layer4_bias)
        # DROPOUT LAYER4
        hidden_drop_4 = tf.nn.dropout(hidden_4, keep_prob)
        
        return tf.matmul(hidden_drop_4, tf_layer5_pesi) + tf_layer5_bias
    
    # ESECUZIONE
    logits = model(tf_train_data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            labels=tf_train_target, logits=logits))  
    # REGULARIZZATION
    # N.B. La regolarizzazione si applica solitamente tra reti full-connected.
    regularizers = tf.nn.l2_loss(tf_layer4_pesi) + tf.nn.l2_loss(tf_layer5_pesi)
    loss = tf.reduce_mean(loss + 0.001 * regularizers)
    
    # DECAY
    global_step = tf.Variable(0)  
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, 
                                               global_step, 
                                               100000, 
                                               0.96, 
                                               staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # ADDESTRAMENTO
    train_prediction = tf.nn.softmax(logits)
    # VALIDAZIONE e TEST
    valid_prediction = tf.nn.softmax(model(tf_valid_data))
    test_prediction = tf.nn.softmax(model(tf_test_data))

In [68]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    print('--- Initialized ---')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_target.shape[0] - batch_size)
        
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = train_target[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_data : batch_data, 
                     tf_train_target : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        
        if (step % 5000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, 
                                                          batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), 
                                                           valid_target))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), 
                                                            test_target))
    

--- Initialized ---
Minibatch loss at step 0: 2.572545
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 5000: 0.880083
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 10000: 0.566940
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 15000: 0.957330
Minibatch accuracy: 68.8%
Validation accuracy: 84.9%
Minibatch loss at step 20000: 0.369184
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 25000: 0.538086
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Test accuracy: 92.7%


IN -> CNN -> RELU + POOL -> RESHAPE -> FULL + RELU + DROPOUT -> FULL(OUT)

In [74]:
graph = tf.Graph()

with graph.as_default():
    
    tf_train_data = tf.placeholder(tf.float32,
            shape=(batch_size, image_size, image_size, num_channels))
    tf_train_target = tf.placeholder(tf.float32,
            shape=(batch_size,                         num_labels))
    
    tf_valid_data = tf.constant(valid_data)
    tf_test_data  = tf.constant(test_data)
    
    # Layer1 CNN + RELU + POOL
    dim_layer1 = [patch_size, patch_size, num_channels, 16]
    tf_layer1_pesi = tf.Variable(
                tf.truncated_normal(dim_layer1, stddev=0.1))
    tf_layer1_bias = tf.Variable(tf.zeros([16]))
    
    # Layer3 FULL
    dim_layer3 = [12 * 12 * 16, num_hidden]
    tf_layer3_pesi = tf.Variable(
                tf.truncated_normal(dim_layer3, stddev=0.1))
    tf_layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer5 FULL
    dim_layer5 = [num_hidden, num_labels]
    tf_layer5_pesi = tf.Variable(
                tf.truncated_normal(dim_layer5, stddev=0.1))
    tf_layer5_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    #
    # MODELLO
    # 
    def model(data):
        # Convolutional LAYER1
        conv_1 = tf.nn.conv2d(data, tf_layer1_pesi, 
                              strides=[1, stride, stride, 1], 
                              padding='VALID')
        # RELU LAYER1
        hidden_1 = tf.nn.relu(conv_1 + tf_layer1_bias)
        # Average POOLING LAYER1
        pool_1 = tf.nn.avg_pool(hidden_1, 
                                [1, pool_size, pool_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Full Connected LAYER3
        shape = pool_1.get_shape().as_list()
        reshape = tf.reshape(pool_1, 
                            [shape[0], shape[1] * shape[2] * shape[3]])
        # RELU LAYER3
        hidden_3 = tf.nn.relu(tf.matmul(reshape, 
                                      tf_layer3_pesi) + tf_layer3_bias)
        # DROPOUT LAYER3
        hidden_drop_3 = tf.nn.dropout(hidden_3, keep_prob)
        
        return tf.matmul(hidden_drop_3, tf_layer5_pesi) + tf_layer5_bias
    
    # ESECUZIONE
    logits = model(tf_train_data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            labels=tf_train_target, logits=logits))  
    # REGULARIZZATION
    # N.B. Tra reti full-connected.
    regularizers = tf.nn.l2_loss(tf_layer5_pesi)
    loss = tf.reduce_mean(loss + 0.001 * regularizers)
    
    # DECAY
    global_step = tf.Variable(0)  
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, 
                                               global_step, 
                                               100000, 
                                               0.96, 
                                               staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(
                                    learning_rate).minimize(loss)
    # ADDESTRAMENTO
    train_prediction = tf.nn.softmax(logits)
    # VALIDAZIONE e TEST
    valid_prediction = tf.nn.softmax(model(tf_valid_data))
    test_prediction = tf.nn.softmax(model(tf_test_data))
    

In [75]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    print('--- Initialized ---')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_target.shape[0] - batch_size)
        
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = train_target[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_data : batch_data, 
                     tf_train_target : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        
        if (step % 5000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, 
                                                          batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), 
                                                           valid_target))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), 
                                                            test_target))
    

--- Initialized ---
Minibatch loss at step 0: 3.047375
Minibatch accuracy: 6.2%
Validation accuracy: 8.5%
Minibatch loss at step 5000: 0.526570
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 10000: 0.636094
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 15000: 0.751565
Minibatch accuracy: 68.8%
Validation accuracy: 84.5%
Minibatch loss at step 20000: 0.400297
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 25000: 0.749457
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Test accuracy: 92.0%


Per ricordare i task di creazione e esecuzione fondamentali

In [39]:
graph = tf.Graph()
with graph.as_default():
    
    size_data = [16,28,28,1]
    tf_data_train = tf.placeholder(tf.float32, shape=(16,28,28,1))
    size_label = [16,10]
    tf_label_train = tf.placeholder(tf.float32, shape=(16,10))
    
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)
    
    def model(data):
        
        #SUPPORT x LAYERS 1,2,3
        size_window = [5,5,1,32]
        tf_pesi_layer1 = tf.Variable(tf.truncated_normal(size_window, stddev=0.1))
        tf_bias_layer1 = tf.Variable(tf.zeros(32))
        
        #LAYER1 CNN
        layer1_cnn = tf.nn.conv2d(data, tf_pesi_layer1, strides=[1,1,1,1], padding='VALID')
        
        #HIDDEN2 RELU
        layer2_relu = tf.nn.relu(layer1_cnn + tf_bias_layer1)
        
        #HIDDEN3 POOL
        size_pool = [1,2,2,1]
        layer3_pool = tf.nn.avg_pool(layer2_relu, size_pool, strides=[1,2,2,1], padding='VALID')
        
        #HIDDEN4 RESHAPE
        shape = layer3_pool.get_shape().as_list()
        layer4_reshape = tf.reshape(layer3_pool, [shape[0], shape[1]*shape[2]*shape[3]])
        
        #SUPPORT x LAYERS 5,6,7
        size_full = [12*12*32,64]
        tf_pesi_layer5 = tf.Variable(tf.truncated_normal(size_full, stddev=0.1))
        tf_bias_layer5 = tf.Variable(tf.zeros(64))
        
        #LAYER5 FULL
        layer5_full = tf.matmul(layer4_reshape, tf_pesi_layer5)
        
        #HIDDEN6 RELU
        layer6_relu = tf.nn.relu(layer5_full + tf_bias_layer5)
        
        #HIDDEN7 DROPOUT
        layer7_dropout = tf.nn.dropout(layer6_relu, 0.5)
        
        #SUPPORT x LAYER 8
        size_final = [64,10]
        tf_pesi_layer8 = tf.Variable(tf.truncated_normal(size_final, stddev=0.1))
        tf_bias_layer8 = tf.Variable(tf.zeros(10))
        
        #LAYER8 FINAL
        layer8_full = tf.matmul(layer7_dropout, tf_pesi_layer8)
        
        return layer8_full + tf_bias_layer8
    
    logits = model(tf_data_train)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_label_train, logits=logits))    
    
    # Decaying learning rate
    global_step = tf.Variable(0)  
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_data))
    test_prediction = tf.nn.softmax(model(tf_test_data))

In [41]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(1000):
        offset = (step * batch_size) % (train_target.shape[0] - batch_size)
        
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = train_target[offset:(offset + batch_size), :]
        
        feed_dict = {tf_data_train : batch_data, 
                     tf_label_train : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_target))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_target))

Minibatch loss at step 0: 2.285804
Minibatch accuracy: 12.5%
Validation accuracy: 12.8%
Minibatch loss at step 500: 1.083577
Minibatch accuracy: 56.2%
Validation accuracy: 12.0%
Test accuracy: 11.0%
